# Predicting Student Admissions with Neural Networks
In this notebook, we predict student admissions to graduate school at UCLA based on three pieces of data:

    -GRE Scores (Test)
    -GPA Scores (Grades)
    -Class rank (1-4)

The dataset originally came from here: http://www.ats.ucla.edu/

### Dependencies

In [1]:
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Neural Network Class

In [43]:
# This calss handles all the procedure from loading data to testing accuracy
class NN:
    # constructor to load data from csv file
    def __init__(self, file):
        np.random.seed(20)  # to make sure, random is most random all the time
        self.__data = pd.read_csv(file)
        print('Data Loaded')
    
    
    # show data
    def getData(self):
        return self.__data
    
    
    # One-hot encode
    # General function to apply one-hot encode on any data, just pass the dataset and col to convert to one-hot encoded vector
    def one_hot_encoder(self, col):
        classes = np.sort(self.__data[col].unique())
        one_hot_vector = np.array([[0  if val != c else 1 for c in classes] for val in self.__data[col]])
        self.__data = pd.merge(
            self.__data,
            pd.DataFrame(data=one_hot_vector, columns=classes),
            left_index=True, 
            right_index=True
        )
        self.__data.drop([col], axis=1, inplace=True)
        print('Done! One Hot Encoding on {} column'.format(col))
    
    
    # Scaling the data
    # We notice that the range for grades is 1.0-4.0, 
    # whereas the range for test scores is roughly 200-800, which is much larger. 
    # This means our data is skewed, and that makes it hard for a neural network to handle. 
    # Let's fit our two features into a range of 0-1, by dividing the grades by 4.0, and the test score by 800.
    # But below will follow for any kind of data
    def scale_data(self, *cols):
        for col in cols:
            self.__data[col] = self.__data[col]/np.max(self.__data[col])
        print('Scaling Done')
    
    
    # Splitting the data into training and testing 
    # further splitting the data into features(X) and lables(y)
    # As we want random values from dataset to be selected as sample, 
    # we will use np.random.choice() to select random indices for sample
    def train_test_split(self, label_col):
        self.__output_neurons = len(label_col)
        sample = np.random.choice(self.__data.index, size=int(len(self.__data)*0.8), replace=False)
        train_data, test_data = self.__data.iloc[sample], self.__data.drop(sample)
        print('No. of rows in training data: ', len(train_data))
        print('No. of rows in testing data: ', len(test_data))
        self.__features = train_data.drop(label_col, axis=1)
        self.__labels = np.array(train_data[label_col])
        self.__features_test = test_data.drop(label_col, axis=1)
        self.__labels_test = test_data[label_col]
    
    
    # activation function
    # previous activation is inputs and weights is current thetas
    # x = np.dot(inputs, weight)
    # or x = np.dot(previous_activation, current_thetas)
    def __sigmoid(self, x):
        return 1/(1+np.exp(-x))
    
    
    # Average negative log likelihood loss function
    def __loss(self, index):
        return -((np.matmul(self.__labels.T, self.__activations[-1])) + ((np.matmul(1-self.__labels.T, 1-self.__activations[-1]))))
    
    
    # derivative of loss function in forward pass
    def __delForward(self, x, y):
        x = x[:, None]
        y = y[None, :]
        return np.matmul(x, y*(1-y))
    
    
    # Define layers of neural network
    # in neurons enter a list 
    # e.g. [input_features/neurons, hidden_layer_1_neurons, hidden_layer_2_neurons, output_layer_neurons]
    def layers(self, neurons):
        self.__neurons = neurons
        print('Layers of Neural Network:')
        print('Layer 1, Input Layer Neurons: {}'.format(self.__neurons[0]))
        for i in range(1, len(self.__neurons)-1):
            print('Layer {}, Hidden Layer {} Neurons: {}'.format(i+1, i, self.__neurons[i]))
        print('Layer {}, Output Layer Neurons: {}'.format(len(self.__neurons), self.__neurons[-1]))
        
    
    # forward pass
    def __forward(self):
        self.__del_forward = []
        for i in range(len(self.__neurons)-2):
            self.__activations.append(self.__sigmoid(np.dot(self.__activations[i], self.__weights[i+1])))
            self.__del_forward.append(self.__delForward(self.__activations[i], self.__activations[i+1]))
    
    
    # reverse/backward pass
    def __backward(self):
        self.__del_backward = []
        for i in range(len(self.__neurons)-2, 0, -1):
            self.__del_backward.append(self.__weights[i+1], self.__del_backward[i+1])
        
    
    # update weights
    def __updateWeights(self):
        for i in range(len(self.__neurons)):
            weights[i] -= aplha*(self.__del_forward*sel.__del_backward.T) 
        
    
    # return weights
    def getWeights(self):
        return self.__weights
    
    
    # Training Neural Network
    def train_nn(self, epochs=1000, alpha=0.1, batch_size=64):
        # n_records, n_features = features.shape
        last_loss = None
        self.__epochs = []
        self.__loss = []
        
        # initialze weights/thetas for all the layers
        self.__weights = [] 
        for i in range(len(self.__neurons)-1):
            self.__weights.append(np.random.normal(scale=self.__neurons[i]**-.5, size=(self.__neurons[i], self.__neurons[i+1])))
        
        for e in range(epochs):
            # indices = np.random.randint(0, self.__features.shape[0], size=batch_size)
            indices = np.random.randint(0, self.__features.shape[0])
            
            # forward
            self.__activations = [np.array(self.__features.iloc[indices])]
            self.__forward()
            
            # backward
            self.__E = np.mean(self.__activations[-1] - labels[indices])
            self.__del_backward = [self.__E, self.__E*self.__weights[-1]]
            self.__backward()
            self.__updateWeights()
            
            # loss
            self.__epochs.append(e)
            self.__loss.append(__loss(indices))
            
            if e % (epochs/10) == 0:
                print('Epoch:', e)
                if e == 0:
                    last_loss = self.__loss[e]
                if last_loss < self.__loss[e]:
                    print('Train loss:', self.__loss[e], "WARNING - Loss Increasing")
                else:
                    print('Train loss:', self.__loss[e])
                last_loss = self.__loss[e]
                print("============")
        print('Training Finished')
        
    
    # Test - Accuracy
    def accuracy(self):
        test_output = sigmoid(np.dot(self.__features_test, self.__weights))
        predictions = test_output > 0.5
        accuracy = np.mean(predictions == self.__labels_test)
        print("Prediction accuracy: {:.3f}".format(accuracy))
        
    
    # Plot loss vs iterations
    def plot(self):
        plt.plot(self.__epochs, self.__loss)

In [44]:
obj = NN('student_data.csv')
obj.one_hot_encoder('rank')
obj.scale_data(['gre', 'gpa'])
obj.train_test_split(['admit'])
obj.layers([6, 6, 6, 1])

Data Loaded
Done! One Hot Encoding on rank column
Scaling Done
No. of rows in training data:  320
No. of rows in testing data:  80
Layers of Neural Network:
Layer 1, Input Layer Neurons: 6
Layer 2, Hidden Layer 1 Neurons: 6
Layer 3, Hidden Layer 2 Neurons: 6
Layer 4, Output Layer Neurons: 1


In [45]:
obj.train_nn()

[0.8   0.965 0.    0.    1.    0.   ]
[0.41041316 0.54168638 0.28191461 0.6942011  0.40753984 0.54404611]


NameError: name 'labels' is not defined

IndexError: list index out of range

12

In [9]:
_.shape

(1, 4)